In [38]:
from pprint import pprint
from typing import cast
from pathlib import Path
from one.api import ONE, OneAlyx
from data_transform import SmbClient

# Login to ONE

In [39]:
USER_NAME = input("Enter your username: ")
PASSWORD = input("Enter your password: ")

one = ONE(username=USER_NAME, password=PASSWORD) # Login to Alyx

# Login to Data Server

In [40]:
DATA_USER_NAME = input("Enter your data server username: ")
DATA_PASSWORD = input("Enter your data server password: ")

smbclient = SmbClient(r"172.16.102.77", "cog_platform", DATA_USER_NAME, DATA_PASSWORD) # Login to Data Server

In [41]:
ROOT_EXPERIMENTAL_FOLDER = Path.cwd() / "data" # Create a directory to store data
pprint(ROOT_EXPERIMENTAL_FOLDER)

WindowsPath('d:/projects/engineering/alyx_data_tianming/data')


# Index experiments in alyx database

In [42]:
one = cast(OneAlyx, one)

# Get all sessions
sessions =  one.alyx.rest('sessions', 'list', no_cache=True)

# Filter for sessions with subject 'omegaL'
eids = [session['id'] for session in sessions if session['subject'] == 'omegaL']
pprint(eids)

['27c8af19-6ec6-4006-97c4-0c7592e67f81', '36560ed6-f5f2-4920-800d-baace226c692']


# Download data

In [43]:
data_directory_paths = [
    ROOT_EXPERIMENTAL_FOLDER.joinpath(*one.eid2path(eid).parts[-5:]) for eid in eids
]

for data_directory_path in data_directory_paths:
    smbclient.download(data_directory_path / "alf", Path.cwd() / "data")


100%|██████████| 388M/388M [00:05<00:00, 73.0MB/s] 
100%|██████████| 431M/431M [00:05<00:00, 78.1MB/s] 


In [44]:
# data_paths is all the paths to the data files we need to load
data_paths = [
    ROOT_EXPERIMENTAL_FOLDER.joinpath(*one.eid2path(eid).parts[-5:]).joinpath(file)  # type: ignore
    for eid in eids
    for file in one.list_datasets(eid)
]
pprint(data_paths)


[WindowsPath('d:/projects/engineering/alyx_data_tianming/data/YangLab/Subjects/omegaL/2021-08-06/001/alf/q_inf_merge.weight_dynamic_res.pkl'),
 WindowsPath('d:/projects/engineering/alyx_data_tianming/data/YangLab/Subjects/omegaL/2021-08-01/001/alf/q_inf_merge.weight_dynamic_res.pkl')]


# Load Data
load data to memory

In [45]:
import pickle
import pandas as pd


def load_data(data_path):
    try:
        with open(data_path, "rb") as f:
            return pickle.load(f)
    except Exception as e:
        print(f"Error loading {data_path}: {e}")
        return None


# all data
datas: list[list] = [
    data for data in (load_data(path) for path in data_paths) if data is not None
]

# Example of how to use the data

In [46]:
from dataclasses import dataclass


@dataclass
class SampleData:
    info: str
    data_df: pd.DataFrame
    data_series: pd.Series
    data_list: list


datas_convert = [[SampleData(*item) for item in data] for data in datas]


datas_convert[0][0].data_df.head()

,index,Unnamed: 0,Unnamed: 0.1,Index,DayTrial,Step,pacmanPos,ghost1Pos,ghost2Pos,ifscared1,...,next_pacman_dir_fill,global_Q_norm,local_Q_norm,evade_blinky_Q_norm,evade_clyde_Q_norm,approach_Q_norm,energizer_Q_norm,weight,contribution,is_correct
0,0,0,0,0,1-1-omegaL-06-Aug-2021-1,1,"(14, 27)","(14, 18)","(15, 19)",1,...,NaN,"[1.0, 0.5681818181818182, -inf, -inf]","[1.0, 0.0, -inf, -inf]","[1.0, 1.0, -inf, -inf]","[1.0, 1.0, -inf, -inf]","[0.0, 0.0, -inf, -inf]","[0.0, 0.0, -inf, -inf]","[0.0, 1.645787633111234, 4.778009788622568e-06...","[0.0, 0.1191543256713139, 3.459258794757151e-0...",NaN
1,1,1,1,1,1-1-omegaL-06-Aug-2021-1,2,"(14, 27)","(14, 18)","(15, 19)",1,...,left,"[1.0, 0.5681818181818182, -inf, -inf]","[1.0, 0.0, -inf, -inf]","[1.0, 1.0, -inf, -inf]","[1.0, 1.0, -inf, -inf]","[0.0, 0.0, -inf, -inf]","[0.0, 0.0, -inf, -inf]","[0.0, 1.645787633111234, 4.778009788622568e-06...","[0.0, 0.1191543256713139, 3.459258794757151e-0...",1.0
2,2,2,2,2,1-1-omegaL-06-Aug-2021-1,3,"(14, 27)","(14, 18)","(15, 19)",1,...,left,"[1.0, 0.5681818181818182, -inf, -inf]","[1.0, 0.0, -inf, -inf]","[1.0, 1.0, -inf, -inf]","[1.0, 1.0, -inf, -inf]","[0.0, 0.0, -inf, -inf]","[0.0, 0.0, -inf, -inf]","[0.0, 1.645787633111234, 4.778009788622568e-06...","[0.0, 0.1191543256713139, 3.459258794757151e-0...",1.0
3,3,3,3,3,1-1-omegaL-06-Aug-2021-1,4,"(14, 27)","(14, 18)","(15, 19)",1,...,left,"[1.0, 0.5681818181818182, -inf, -inf]","[1.0, 0.0, -inf, -inf]","[1.0, 1.0, -inf, -inf]","[1.0, 1.0, -inf, -inf]","[0.0, 0.0, -inf, -inf]","[0.0, 0.0, -inf, -inf]","[0.0, 1.645787633111234, 4.778009788622568e-06...","[0.0, 0.1191543256713139, 3.459258794757151e-0...",1.0
4,4,4,4,4,1-1-omegaL-06-Aug-2021-1,5,"(14, 27)","(14, 18)","(15, 19)",1,...,left,"[1.0, 0.5681818181818182, -inf, -inf]","[1.0, 0.0, -inf, -inf]","[1.0, 1.0, -inf, -inf]","[1.0, 1.0, -inf, -inf]","[0.0, 0.0, -inf, -inf]","[0.0, 0.0, -inf, -inf]","[0.0, 1.645787633111234, 4.778009788622568e-06...","[0.0, 0.1191543256713139, 3.459258794757151e-0...",1.0
